In [ ]:
import sys
sys.path.append("../")
import copy

from dotenv import load_dotenv
load_dotenv()
import geopandas as gpd
import importlib
import contextily as cx
import gtfs_kit as gk
import fastsim as fsim
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
from pathlib import Path
from rasterio.plot import show
import seaborn as sns

from openbustools import plotting, spatial, standardfeeds
from openbustools.traveltime import data_loader, model_utils
from openbustools.drivecycle import trajectory, busnetwork
from openbustools.drivecycle.physics import conditions, energy, vehicle

In [ ]:
epsg = 32148
timezone = "America/Los_Angeles"
coord_ref_center = [386910,69022]
chop_n = 500
point_sep_m = 200
dem_file = Path("..","data","kcm_spatial","usgs10m_dem_32148.tif")
phone_trajectory_folders = [x for x in Path("..","data","kcm_sensor","match_realtime","gnss_validated").glob("*") if x.is_dir()]
realtime_folder = Path("..","data","kcm_sensor_realtime")
gnss_solution_file = Path("..","data","gnss","CEErover_solution_20240312160029.LLH")

veh = fsim.vehicle.Vehicle.from_vehdb(63, veh_file=Path("..", "data", "FASTSim_py_veh_db.csv")) # New Flyer XE40
# veh_mod = copy.deepcopy(veh)
# veh_mod.max_regen = 0.01
# veh = veh_mod

model_folder = "../logs"
run_label = "/kcm"
model_type = "GRU"
fold_num = 0
version = "version_0"
model = model_utils.load_model(model_folder, run_label, model_type, fold_num, version=version)
model.eval()

### Altoona, FastSIM Standardized Cycle Comparison (validate BEB model)

In [ ]:
altoona_economy = {
    "manhattan.csv": (2767 / 1000),
    "orange_county.csv": (2176 / 1000),
    "hd_udds.csv": (1980 / 1000)
}
consumptions = []
for cycle_file in ["manhattan.csv", "orange_county.csv", "hd_udds.csv"]:
    df = pd.read_csv(Path("..","data",cycle_file))
    cycle = {
        "cycSecs": df["Time (seconds)"].to_numpy(),
        "cycMps": df["Speed (mph)"].to_numpy() * 0.44704,
        "cycGrade": np.zeros(df.shape[0]),
        "cycRoadType": np.zeros(df.shape[0])
    }
    cycle = fsim.cycle.Cycle.from_dict(cycle)
    sim_drive = fsim.simdrive.SimDrive(cycle, veh)
    sim_drive.sim_drive()
    consumptions.append(sim_drive.battery_kwh_per_mi)
for i,cycle_file in enumerate(["manhattan.csv", "orange_county.csv", "hd_udds.csv"]):
    print(f"{cycle_file}: (FASTSim BEB): {consumptions[i]:.2f} kWh/mi | (Altoona: {altoona_economy[cycle_file]:.2f} kWh/mi)")

In [ ]:
# Smooth or increase velocity peaks
df['Speed (mph)_smoothed'] = spatial.apply_sg_filter(df["Speed (mph)"].to_numpy(), clip_min=0, clip_max=80)
df['Speed (mph)_peaked_2'] = spatial.apply_peak_filter(df['Speed (mph)'].to_numpy(), acc_scalar=2.0, dec_scalar=2.0, window_len=51, clip_min=0, clip_max=80)
df['Speed (mph)_peaked_5'] = spatial.apply_peak_filter(df['Speed (mph)'].to_numpy(), acc_scalar=5.0, dec_scalar=5.0, window_len=51, clip_min=0, clip_max=80)

fig, axes = plt.subplots(2, 1, figsize=(8, 5))
axes = axes.flatten()

# Speed profiles
sns.lineplot(x=df["Time (seconds)"], y=df["Speed (mph)_peaked_2"], ax=axes[0])
sns.lineplot(x=df["Time (seconds)"], y=df["Speed (mph)_peaked_5"], ax=axes[0])
sns.lineplot(x=df["Time (seconds)"], y=df["Speed (mph)"], ax=axes[0])
sns.lineplot(x=df["Time (seconds)"], y=df["Speed (mph)_smoothed"], ax=axes[0])
axes[0].set_xlabel("Time (seconds)")
axes[0].set_ylabel("Speed (mph)")

# Acceleration histograms
df['acc_mph_s'] = df['Speed (mph)'].diff() / df['Time (seconds)'].diff()
df['acc_mph_s_smoothed'] = df['Speed (mph)_smoothed'].diff() / df['Time (seconds)'].diff()
df['acc_mph_s_peaked_2'] = df['Speed (mph)_peaked_2'].diff() / df['Time (seconds)'].diff()
df['acc_mph_s_peaked_5'] = df['Speed (mph)_peaked_5'].diff() / df['Time (seconds)'].diff()
sns.histplot(df['acc_mph_s_peaked_2'], bins=100, kde=True, ax=axes[1], label="Boosted 2x")
sns.histplot(df['acc_mph_s_peaked_5'], bins=100, kde=True, ax=axes[1], label="Boosted 5x")
sns.histplot(df['acc_mph_s'], bins=100, kde=True, ax=axes[1], label="Original")
sns.histplot(df['acc_mph_s_smoothed'], bins=100, kde=True, ax=axes[1], label="Smoothed")
axes[1].set_xlim(-10, 10)
axes[1].set_ylim(0, 120)
axes[1].set_xlabel("Acceleration (mph/s)")
axes[1].legend()

fig.suptitle("Comparison of Speed Filtering Methods (HD-UDDS Cycle)")
fig.tight_layout()
plt.show()
fig.savefig(Path("..","plots","speed_filtering.png"))

### Phone, Receiver, Realtime Comparison (validate realtime trajectory)

In [ ]:
# Load phone/realtime data as trajectories
phone_trajectories = [standardfeeds.get_phone_trajectory(x, timezone=timezone, epsg=epsg, coord_ref_center=coord_ref_center, dem_file=dem_file, chop_n=chop_n, resample=True) for x in phone_trajectory_folders]
phone_gnss_trajectories = [standardfeeds.get_gnss_trajectory(x, gnss_solution_file=gnss_solution_file, resample=True) for x in phone_trajectories]
phone_realtime_trajectories = [standardfeeds.get_realtime_trajectory(x, realtime_folder=realtime_folder,  resample=True) for x in phone_trajectories]
print(len(phone_trajectories[0].gdf))
print(len(phone_gnss_trajectories[0].gdf))
print(len(phone_realtime_trajectories[0].gdf))

In [ ]:
plot_df = []
for i, df in enumerate([phone_gnss_trajectories[0].gdf, phone_gnss_trajectories[1].gdf, phone_gnss_trajectories[2].gdf]):
    df['traj_num'] = i
    plot_df.append(df)
plot_df = pd.concat(plot_df)
plot_df['calc_speed_m_s'] = np.clip(plot_df['calc_speed_m_s'].to_numpy(), 0, 30) * 2.23694

fig, axes = plt.subplots(1,3,figsize=(10,7.5))
axes = axes.flatten()

metrics = ["calc_speed_m_s", "calc_elev_m", "cumul_time_s"]
ax_titles = ["Speed (mph)", "Elevation (m)", "Cumulative Time (s)"]
for i, df in enumerate([phone_gnss_trajectories[0].gdf, phone_gnss_trajectories[1].gdf, phone_gnss_trajectories[2].gdf]):
    plot_df.plot(ax=axes[i], column=metrics[i], markersize=1, legend=True)
    axes[i].set_xticks([])
    axes[i].set_yticks([])
    axes[i].set_title(ax_titles[i])
    # cx.add_basemap(ax=axes[i], crs=phone_gnss_trajectories[i].gdf.crs.to_string(), alpha=0.4, source=cx.providers.MapBox(accessToken=os.getenv(key="MAPBOX_TOKEN")))
fig.suptitle("Validation Trajectories Recorded with Phone and GNSS Receiver", horizontalalignment='center', verticalalignment='top')
fig.tight_layout()
fig.savefig(Path("..","plots","gnss_maps.png"))
plt.show()

In [ ]:
plot_res_smoothed = []

consumptions_smoothed = []
signal_errors_smoothed = []

consumptions_clipped = []
signal_errors_clipped = []

# ONLY CLIP
for i, (traj_phone, traj_gnss, traj_realtime) in enumerate(zip(phone_trajectories, phone_gnss_trajectories, phone_realtime_trajectories)):
    # Energy analysis for phone
    cycle_phone = {
        "cycGrade": np.clip(spatial.divide_fwd_back_fill(np.diff(traj_phone.gdf['calc_elev_m'], prepend=traj_phone.gdf['calc_elev_m'].iloc[0]), traj_phone.gdf['calc_dist_m']), -0.15, 0.15),
        "mps": np.clip(traj_phone.gdf["calc_speed_m_s"].to_numpy(), 0, 30),
        "time_s": traj_phone.gdf['cumul_time_s'].to_numpy(),
        "road_type": np.zeros(len(traj_phone.gdf))
    }
    cycle_phone = fsim.cycle.Cycle.from_dict(fsim.cycle.resample(cycle_phone, new_dt=1))
    sim_drive_phone = fsim.simdrive.SimDrive(cycle_phone, veh)
    sim_drive_phone.sim_drive()

    # Energy analysis for GNSS
    cycle_gnss = {
        "cycGrade": np.clip(spatial.divide_fwd_back_fill(np.diff(traj_gnss.gdf['calc_elev_m'], prepend=traj_gnss.gdf['calc_elev_m'].iloc[0]), traj_gnss.gdf['calc_dist_m']), -0.15, 0.15),
        "mps": np.clip(traj_gnss.gdf["calc_speed_m_s"].to_numpy(), 0, 30),
        "time_s": traj_gnss.gdf['cumul_time_s'].to_numpy(),
        "road_type": np.zeros(len(traj_gnss.gdf))
    }
    cycle_gnss = fsim.cycle.Cycle.from_dict(fsim.cycle.resample(cycle_gnss, new_dt=1))
    sim_drive_gnss = fsim.simdrive.SimDrive(cycle_gnss, veh)
    sim_drive_gnss.sim_drive()

    # Energy analysis for realtime
    cycle_realtime = {
        "cycGrade": np.clip(spatial.divide_fwd_back_fill(np.diff(traj_realtime.gdf['calc_elev_m'], prepend=traj_realtime.gdf['calc_elev_m'].iloc[0]), traj_realtime.gdf['calc_dist_m']), -0.15, 0.15),
        "mps": np.clip(traj_realtime.gdf["calc_speed_m_s"].to_numpy(), 0, 30),
        "time_s": traj_realtime.gdf['cumul_time_s'].to_numpy(),
        "road_type": np.zeros(len(traj_realtime.gdf))
    }
    cycle_realtime = fsim.cycle.Cycle.from_dict(fsim.cycle.resample(cycle_realtime, new_dt=1))
    sim_drive_realtime = fsim.simdrive.SimDrive(cycle_realtime, veh)
    sim_drive_realtime.sim_drive()
    consumptions_clipped.append((sim_drive_phone.electric_kwh_per_mi, sim_drive_gnss.electric_kwh_per_mi, sim_drive_realtime.electric_kwh_per_mi))
    signal_errors_clipped.append((spatial.eval_signal_error(sim_drive_phone.cyc.mps, sim_drive_phone.cyc.time_s, sim_drive_realtime.cyc.mps, sim_drive_realtime.cyc.time_s), spatial.eval_signal_error(sim_drive_phone.cyc.mps, sim_drive_phone.cyc.time_s, sim_drive_gnss.cyc.mps, sim_drive_gnss.cyc.time_s)))

# SG FILTER THEN CLIP
for i, (traj_phone, traj_gnss, traj_realtime) in enumerate(zip(phone_trajectories, phone_gnss_trajectories, phone_realtime_trajectories)):
    # Energy analysis for phone
    cycle_phone = {
        "cycGrade": np.clip(spatial.divide_fwd_back_fill(np.diff(traj_phone.gdf['calc_elev_m'], prepend=traj_phone.gdf['calc_elev_m'].iloc[0]), traj_phone.gdf['calc_dist_m']), -0.15, 0.15),
        "mps": spatial.apply_sg_filter(traj_phone.gdf["calc_speed_m_s"].to_numpy(), clip_min=0, clip_max=30),
        "time_s": traj_phone.gdf['cumul_time_s'].to_numpy(),
        "road_type": np.zeros(len(traj_phone.gdf))
    }
    cycle_phone = fsim.cycle.Cycle.from_dict(fsim.cycle.resample(cycle_phone, new_dt=1))
    sim_drive_phone = fsim.simdrive.SimDrive(cycle_phone, veh)
    sim_drive_phone.sim_drive()

    # Energy analysis for GNSS
    cycle_gnss = {
        "cycGrade": np.clip(spatial.divide_fwd_back_fill(np.diff(traj_gnss.gdf['calc_elev_m'], prepend=traj_gnss.gdf['calc_elev_m'].iloc[0]), traj_gnss.gdf['calc_dist_m']), -0.15, 0.15),
        "mps": spatial.apply_sg_filter(traj_gnss.gdf["calc_speed_m_s"].to_numpy(), clip_min=0, clip_max=30),
        "time_s": traj_gnss.gdf['cumul_time_s'].to_numpy(),
        "road_type": np.zeros(len(traj_gnss.gdf))
    }
    cycle_gnss = fsim.cycle.Cycle.from_dict(fsim.cycle.resample(cycle_gnss, new_dt=1))
    sim_drive_gnss = fsim.simdrive.SimDrive(cycle_gnss, veh)
    sim_drive_gnss.sim_drive()

    # Energy analysis for realtime
    cycle_realtime = {
        "cycGrade": np.clip(spatial.divide_fwd_back_fill(np.diff(traj_realtime.gdf['calc_elev_m'], prepend=traj_realtime.gdf['calc_elev_m'].iloc[0]), traj_realtime.gdf['calc_dist_m']), -0.15, 0.15),
        "mps": spatial.apply_sg_filter(traj_realtime.gdf["calc_speed_m_s"].to_numpy(), clip_min=0, clip_max=30),
        "time_s": traj_realtime.gdf['cumul_time_s'].to_numpy(),
        "road_type": np.zeros(len(traj_realtime.gdf))
    }
    cycle_realtime = fsim.cycle.Cycle.from_dict(fsim.cycle.resample(cycle_realtime, new_dt=1))
    sim_drive_realtime = fsim.simdrive.SimDrive(cycle_realtime, veh)
    sim_drive_realtime.sim_drive()

    plot_res_smoothed.append((sim_drive_phone, sim_drive_gnss, sim_drive_realtime))
    consumptions_smoothed.append((sim_drive_phone.electric_kwh_per_mi, sim_drive_gnss.electric_kwh_per_mi, sim_drive_realtime.electric_kwh_per_mi))
    signal_errors_smoothed.append((spatial.eval_signal_error(sim_drive_phone.cyc.mps, sim_drive_phone.cyc.time_s, sim_drive_realtime.cyc.mps, sim_drive_realtime.cyc.time_s), spatial.eval_signal_error(sim_drive_phone.cyc.mps, sim_drive_phone.cyc.time_s, sim_drive_gnss.cyc.mps, sim_drive_gnss.cyc.time_s)))

phone_consumptions_clipped = [x[0] for x in consumptions_clipped]
gnss_consumptions_clipped = [x[1] for x in consumptions_clipped]
realtime_consumptions_clipped = [x[2] for x in consumptions_clipped]

phone_consumptions_smoothed = [x[0] for x in consumptions_smoothed]
gnss_consumptions_smoothed = [x[1] for x in consumptions_smoothed]
realtime_consumptions_smoothed = [x[2] for x in consumptions_smoothed]

rmse_phone_rt_clip = np.sqrt(np.mean([(x-y)**2 for x,y in zip(phone_consumptions_clipped, realtime_consumptions_clipped)]))
rmse_phone_gnss_clip = np.sqrt(np.mean([(x-y)**2 for x,y in zip(phone_consumptions_clipped, gnss_consumptions_clipped)]))
rmse_phone_rt_smooth = np.sqrt(np.mean([(x-y)**2 for x,y in zip(phone_consumptions_smoothed, realtime_consumptions_smoothed)]))
rmse_phone_gnss_smooth = np.sqrt(np.mean([(x-y)**2 for x,y in zip(phone_consumptions_smoothed, gnss_consumptions_smoothed)]))

print(f"Clipped signal RMSE (Phone/Realtime): {np.mean([x[0] for x in signal_errors_clipped])}")
print(f"Smoothed signal RMSE (Phone/Realtime): {np.mean([x[0] for x in signal_errors_smoothed])}")
print()
print(f"Clipped consumption RMSE (Phone/Realtime): {rmse_phone_rt_clip}")
print(f"Smoothed consumption RMSE (Phone/Realtime): {rmse_phone_rt_smooth}")
print()
print(f"Percent Reduction w/Smoothing (Phone/Realtime signal): {100*(1-np.mean([x[0] for x in signal_errors_smoothed])/np.mean([x[0] for x in signal_errors_clipped]))}")
print(f"Percent Reduction w/Smoothing (Phone/Realtime consumption): {100*(1-rmse_phone_rt_smooth/rmse_phone_rt_clip)}")
print()
print()
print(f"Clipped signal RMSE (Phone/GNSS): {np.mean([x[1] for x in signal_errors_clipped])}")
print(f"Smoothed signal RMSE (Phone/GNSS): {np.mean([x[1] for x in signal_errors_smoothed])}")
print()
print(f"Clipped consumption RMSE (Phone/GNSS): {rmse_phone_gnss_clip}")
print(f"Smoothed consumption RMSE (Phone/GNSS): {rmse_phone_gnss_smooth}")
print()
print(f"Percent Reduction w/Smoothing (signal): {100*(1-np.mean([x[1] for x in signal_errors_smoothed])/np.mean([x[1] for x in signal_errors_clipped]))}")
print(f"Percent Reduction w/Smoothing (consumption): {100*(1-rmse_phone_gnss_smooth/rmse_phone_gnss_clip)}")

In [ ]:
# Phone/Realtime, Phone/GNSS
print(signal_errors_clipped)
print(signal_errors_smoothed)

In [ ]:
# Phone/GNSS/Realtime
print(consumptions_clipped)
print(consumptions_smoothed)

In [ ]:
fig, axes = plt.subplots(3,2,figsize=(10,8))
for traj_n in range(3):
    ax = axes[traj_n,0]
    ax.legend().remove()
    ax.set_ylabel("Speed (mph)")
    ax.set_xlabel("Time (s)")
    ax.set_ylim(0,70)
    # Completely unprocessed trajectories
    sns.lineplot(x=phone_trajectories[traj_n].gdf["cumul_time_s"], y=phone_trajectories[traj_n].gdf["calc_speed_m_s"]*2.23, ax=ax, label="Phone")
    sns.lineplot(x=phone_gnss_trajectories[traj_n].gdf["cumul_time_s"], y=phone_gnss_trajectories[traj_n].gdf["calc_speed_m_s"]*2.23, ax=ax, label="GNSS")
    sns.lineplot(x=phone_realtime_trajectories[traj_n].gdf["cumul_time_s"], y=phone_realtime_trajectories[traj_n].gdf["calc_speed_m_s"]*2.23, ax=ax, label="GTFS-RT")
for traj_n in range(3):
    ax = axes[traj_n,1]
    ax.set_ylabel("Speed (mph)")
    ax.set_xlabel("Time (s)")
    ax.set_ylim(0,70)
    sim_drive_phone, sim_drive_gnss, sim_drive_realtime = plot_res_smoothed[traj_n]
    # Trajectories post-processed, post-fastsim
    sns.lineplot(x=sim_drive_phone.cyc.time_s, y=sim_drive_phone.cyc.mph, ax=ax, label="Phone")
    sns.lineplot(x=sim_drive_gnss.cyc.time_s, y=sim_drive_gnss.cyc.mph, ax=ax, label="GNSS")
    sns.lineplot(x=sim_drive_realtime.cyc.time_s, y=sim_drive_realtime.cyc.mph, ax=ax, label="GTFS-RT")
axes[0,0].set_title("Input Trip Cycles")
axes[0,1].set_title("Filtered Trip Cycles")

[ax.legend().remove() for ax in axes.flatten()]
axes[0,1].legend()
fig.suptitle(f"Drive Cycles for Validation Trips")
fig.tight_layout()
plt.show()
fig.savefig(Path("..","plots","gnss_cycles.png"))

In [ ]:
all_res = []
for i in range(len(plot_res_smoothed)):
    for j, source in enumerate(['phone','gnss','realtime']):
        rr = plot_res_smoothed[i][j].rr_kw.sum() / 3600
        accel = plot_res_smoothed[i][j].accel_kw.sum() / 3600
        aux = plot_res_smoothed[i][j].aux_in_kw.sum() / 3600
        ascent = plot_res_smoothed[i][j].ascent_kw.sum() / 3600
        loss = plot_res_smoothed[i][j].ess_loss_kw.sum() / 3600
        drag = plot_res_smoothed[i][j].drag_kw.sum() / 3600
        total = sum([rr, accel, aux, ascent, loss, drag])
        res = {
            "Trajectory": i,
            "Source": source.capitalize() if source!='gnss' else source.upper(),
            "Total": total,
            "Rolling": rr,
            "Acceleration": accel,
            "Auxiliary": aux,
            "Ascent": ascent,
            "Loss": loss,
            "Drag": drag
        }
        all_res.append(res)
all_res = pd.DataFrame(all_res)
all_res = all_res.melt(id_vars=['Trajectory','Source'], value_vars=['Rolling','Acceleration','Auxiliary','Ascent','Loss','Drag'], var_name='component', value_name='kWh')
all_res

fig, axes = plt.subplots(3, 1, figsize=(8,7.5))
axes = axes.flatten()
for i in range(len(plot_res_smoothed)):
    sns.barplot(all_res[all_res['Trajectory']==i], x="component", y="kWh", hue="Source", ax=axes[i])
    axes[i].legend().remove()
    axes[i].set_ylim(-10,10)
    axes[i].set_xlabel(None)
    axes[i].set_ylabel(f"Trip {i+1} (kWh)")
axes[0].legend(loc="lower right")
fig.suptitle("Net Energy Consumption for Validation Trips")
fig.tight_layout()
fig.savefig(Path("..","plots","gnss_net_energy.png"))

In [ ]:
fig, axes = plt.subplots(1, 2, gridspec_kw={'width_ratios': [1, 3]}, figsize=(10,7))
axes = axes.flatten()

sns.lineplot(x=phone_trajectories[1].gdf["cumul_time_s"], y=phone_trajectories[1].gdf["calc_elev_m"], label="Phone", ax=axes[1])
sns.lineplot(x=phone_realtime_trajectories[1].gdf["cumul_time_s"], y=phone_realtime_trajectories[1].gdf["calc_elev_m"], label="GTFS-RT", ax=axes[1])
axes[1].set_ylabel("Elevation (m)")
axes[1].set_xlabel("Time (s)")

phone_trajectories[1].gdf.plot(ax=axes[0], markersize=.1)
phone_realtime_trajectories[1].gdf.plot(ax=axes[0], markersize=.1)
axes[0].set_xticks([])
axes[0].set_yticks([])
cx.add_basemap(ax=axes[0], crs=phone_trajectories[1].gdf.crs.to_string(), alpha=0.4, source=cx.providers.MapBox(accessToken=os.getenv(key="MAPBOX_TOKEN")))

fig.tight_layout()
plt.show()
fig.savefig(Path("..","plots","realtime_elev_discrepancy.png"))

In [ ]:
fig, axes = plt.subplots(1,1,figsize=(10,5))

sim_phone, sim_gnss, sim_realtime = plot_res_smoothed[2]
sns.lineplot(x=sim_phone.cyc.time_s, y=sim_phone.ascent_kw.cumsum() / 3600, label="Phone", ax=axes)
sns.lineplot(x=sim_realtime.cyc.time_s, y=sim_realtime.ascent_kw.cumsum() / 3600, label="GTFS-RT", ax=axes)
axes.set_ylabel("Cumulative Ascent Load (kWh)")
axes.set_xlabel("Time (s)")
fig.tight_layout()
plt.show()
fig.savefig(Path("..","plots","realtime_ascent_discrepancy.png"))

In [ ]:
fig, axes = plt.subplots(9,len(plot_res_smoothed),figsize=(18,18))
for traj_n in range(3):
    sim_phone, sim_gnss, sim_realtime = plot_res_smoothed[traj_n]
    ax = axes[0,traj_n]
    sns.lineplot(x=sim_phone.cyc.time_s, y=sim_phone.ess_kw_out_ach.cumsum() / 3600, ax=ax, label="Phone")
    sns.lineplot(x=sim_gnss.cyc.time_s, y=sim_gnss.ess_kw_out_ach.cumsum() / 3600, ax=ax, label="GNSS")
    sns.lineplot(x=sim_realtime.cyc.time_s, y=sim_realtime.ess_kw_out_ach.cumsum() / 3600, ax=ax, label="GTFS-RT")
    ax = axes[1,traj_n]
    sns.lineplot(x=sim_phone.cyc.time_s, y=sim_phone.rr_kw.cumsum() / 3600, ax=ax, label="Phone")
    sns.lineplot(x=sim_gnss.cyc.time_s, y=sim_gnss.rr_kw.cumsum() / 3600, ax=ax, label="GNSS")
    sns.lineplot(x=sim_realtime.cyc.time_s, y=sim_realtime.rr_kw.cumsum() / 3600, ax=ax, label="GTFS-RT")
    ax = axes[2,traj_n]
    sns.lineplot(x=sim_phone.cyc.time_s, y=sim_phone.drag_kw.cumsum() / 3600, ax=ax, label="Phone")
    sns.lineplot(x=sim_gnss.cyc.time_s, y=sim_gnss.drag_kw.cumsum() / 3600, ax=ax, label="GNSS")
    sns.lineplot(x=sim_realtime.cyc.time_s, y=sim_realtime.drag_kw.cumsum() / 3600, ax=ax, label="GTFS-RT")
    ax = axes[3,traj_n]
    sns.lineplot(x=sim_phone.cyc.time_s, y=sim_phone.ascent_kw.cumsum() / 3600, ax=ax, label="Phone")
    sns.lineplot(x=sim_gnss.cyc.time_s, y=sim_gnss.ascent_kw.cumsum() / 3600, ax=ax, label="GNSS")
    sns.lineplot(x=sim_realtime.cyc.time_s, y=sim_realtime.ascent_kw.cumsum() / 3600, ax=ax, label="GTFS-RT")
    ax = axes[4,traj_n]
    sns.lineplot(x=sim_phone.cyc.time_s, y=sim_phone.aux_in_kw.cumsum() / 3600, ax=ax, label="Phone")
    sns.lineplot(x=sim_gnss.cyc.time_s, y=sim_gnss.aux_in_kw.cumsum() / 3600, ax=ax, label="GNSS")
    sns.lineplot(x=sim_realtime.cyc.time_s, y=sim_realtime.aux_in_kw.cumsum() / 3600, ax=ax, label="GTFS-RT")
    ax = axes[5,traj_n]
    sns.lineplot(x=sim_phone.cyc.time_s, y=sim_phone.ess_loss_kw.cumsum() / 3600, ax=ax, label="Phone")
    sns.lineplot(x=sim_gnss.cyc.time_s, y=sim_gnss.ess_loss_kw.cumsum() / 3600, ax=ax, label="GNSS")
    sns.lineplot(x=sim_realtime.cyc.time_s, y=sim_realtime.ess_loss_kw.cumsum() / 3600, ax=ax, label="GTFS-RT")
    ax = axes[6,traj_n]
    sns.lineplot(x=sim_phone.cyc.time_s, y=sim_phone.accel_kw.cumsum() / 3600, ax=ax, label="Phone")
    sns.lineplot(x=sim_gnss.cyc.time_s, y=sim_gnss.accel_kw.cumsum() / 3600, ax=ax, label="GNSS")
    sns.lineplot(x=sim_realtime.cyc.time_s, y=sim_realtime.accel_kw.cumsum() / 3600, ax=ax, label="GTFS-RT")
    ax = axes[7,traj_n]
    sns.lineplot(x=sim_phone.cyc.time_s, y=sim_phone.mph_ach, ax=ax, label="Phone")
    sns.lineplot(x=sim_gnss.cyc.time_s, y=sim_gnss.mph_ach, ax=ax, label="GNSS")
    sns.lineplot(x=sim_realtime.cyc.time_s, y=sim_realtime.mph_ach, ax=ax, label="GTFS-RT")
    ax = axes[8,traj_n]
    sns.lineplot(x=sim_phone.cyc.time_s, y=sim_phone.cyc.grade, ax=ax, label="Phone")
    sns.lineplot(x=sim_gnss.cyc.time_s, y=sim_gnss.cyc.grade, ax=ax, label="GNSS")
    sns.lineplot(x=sim_realtime.cyc.time_s, y=sim_realtime.cyc.grade, ax=ax, label="GTFS-RT")
axes[0,0].set_ylabel("ESS Energy Consumption")
axes[1,0].set_ylabel("Rolling Resistance")
axes[2,0].set_ylabel("Drag")
axes[3,0].set_ylabel("Ascent")
axes[4,0].set_ylabel("Auxiliary")
axes[5,0].set_ylabel("ESS Loss")
axes[6,0].set_ylabel("Acceleration")
axes[7,0].set_ylabel("Speed")
axes[8,0].set_ylabel("Grade")
fig.suptitle("Energy Consumption Components for Validation Trips")
fig.tight_layout()
plt.show()


### Aggregated and Predicted Cycle Comparison

In [ ]:
# Load a static feed and break each shape into regularly spaced points
static_file = Path("..", "data", "kcm_static", "2024_03_08")
static_feed = gk.read_feed(static_file, dist_units="km")
route_shape_points = standardfeeds.segmentize_shapes(static_feed, epsg=epsg, point_sep_m=point_sep_m)

# Load set of realtime data to aggregate to route shapes; join it to static feed ids
realtime_files = [Path("..", "data", "kcm_realtime", "processed", "analysis", f"2024_03_1{x}.pkl") for x in np.arange(7)]
realtime_data = [pd.read_pickle(i) for i in realtime_files]
realtime_data = pd.concat(realtime_data)
realtime_data = pd.merge(realtime_data[['calc_speed_m_s', 'x', 'y', 'trip_id']], static_feed.trips[['trip_id', 'shape_id']], on='trip_id')
realtime_data = gpd.GeoDataFrame(realtime_data, geometry=gpd.points_from_xy(realtime_data.x, realtime_data.y), crs=epsg)

# Group realtime data by shape and find closest static point in that shape for each observation
route_shape_metrics = realtime_data.groupby('shape_id').apply(lambda x: gpd.sjoin_nearest(x, route_shape_points[x.name]), include_groups=False)
route_shape_metrics = route_shape_metrics.drop(columns=['shape_id']).reset_index()
route_shape_metrics = route_shape_metrics.groupby(['shape_id', 'seq_id'], as_index=False).agg({
    'calc_speed_m_s': ['mean', 'std'],
    'trip_id': 'count',
    'geometry': 'first'
})
route_shape_metrics = {
    'geometry': route_shape_metrics[('geometry', 'first')],
    'speed_mean': route_shape_metrics[('calc_speed_m_s', 'mean')],
    'speed_std': route_shape_metrics[('calc_speed_m_s', 'std')],
    'count_n': route_shape_metrics[('trip_id', 'count')],
    'shape_id': route_shape_metrics['shape_id'],
    'seq_id': route_shape_metrics['seq_id']
}
route_shape_metrics = gpd.GeoDataFrame(route_shape_metrics, crs=epsg)
# Drop routes that have less than 10 observations (will vary with chosen point_sep_m)
route_shape_lens = route_shape_metrics.groupby('shape_id', as_index=False).size()
drop_routes = route_shape_lens[route_shape_lens['size'] < 10]['shape_id']
route_shape_metrics = route_shape_metrics[~route_shape_metrics['shape_id'].isin(drop_routes)].copy()

In [ ]:
# Subset of shape_ids for testing
n_sample_routes = 100
n_plot_routes = 3
# good_routes = ['20033002', '11345009', '20345010', '10027004']
# bad_routes = ['10057005', '30165010', '30271024', '10102003']
# plot_routes = bad_routes
plot_routes = pd.Series(route_shape_metrics['shape_id'].unique()).sample(n_sample_routes).to_numpy()
plot_df = route_shape_metrics[route_shape_metrics['shape_id'].isin(plot_routes)].copy()
plot_df_groups = {k: v for k, v in plot_df.groupby('shape_id')}

In [ ]:
# Turning regularly spaced points w/speeds into calculated distances and times
# Important to base distance on sequence id because some points do not have observations
for k,df in plot_df_groups.items():
    plot_df_groups[k]['cumul_dist_m'] = df['seq_id'] * point_sep_m
    plot_df_groups[k]['calc_dist_m'] = df['cumul_dist_m'].diff().fillna(0)
    plot_df_groups[k]['calc_time_s'] = df['calc_dist_m'] / df['speed_mean']
    plot_df_groups[k]['cumul_time_s'] = df['calc_time_s'].cumsum()
    plot_df_groups[k]['speed_std'] = df['speed_std'].bfill().ffill()

In [ ]:
# Overview of the routes
fig, axes = plt.subplots(1, n_plot_routes, figsize=(30,5))
for i, (k,df) in enumerate(plot_df_groups.items()):
    if i < n_plot_routes:
        ax = axes[i]
        ax.plot(df['cumul_dist_m'], df['speed_mean'], label="Mean Speed")
        ax.fill_between(df['cumul_dist_m'], df['speed_mean'] - df['speed_std'], df['speed_mean'] + df['speed_std'], alpha=0.2, label="Speed Std")
        ax.set_xlabel("Distance [m]")
        ax.set_ylabel("Speed [m/s]")
        ax.set_title(f"Shape {k}")
        ax.legend()
fig.tight_layout()
plt.show()

# Map of the routes
fig, axes = plt.subplots(1, n_plot_routes, figsize=(30,5))
for i, (k,df) in enumerate(plot_df_groups.items()):
    if i < n_plot_routes:
        ax = axes[i]
        df.plot(ax=ax, column='speed_mean', cmap='plasma', legend=True)
        cx.add_basemap(ax=ax, crs=plot_df.crs.to_string(), alpha=0.6, source=cx.providers.MapBox(accessToken=os.getenv(key="MAPBOX_TOKEN")))
        ax.set_title(f"Shape {k}")
fig.tight_layout()
plt.show()

In [ ]:
# Create trajectory for each shape
plot_trajectories = []
for shape_id, df in plot_df_groups.items():
    traj = trajectory.Trajectory(
        point_attr={
            "lon": df.to_crs(4326).geometry.x.to_numpy(),
            "lat": df.to_crs(4326).geometry.y.to_numpy(),
            "locationtime": df.cumul_time_s.to_numpy(),
            "measured_speed_m_s": df.speed_mean.to_numpy(),
            "seq_id": df.seq_id.to_numpy(),
            "count_n": df.count_n.to_numpy()
        },
        traj_attr={
            'shape_id': shape_id,
            "coord_ref_center": coord_ref_center,
            "epsg": epsg,
            "dem_file": dem_file,
            "t_min_of_day": 9*60,
            "t_day_of_week": 4,
            "start_epoch": df.cumul_time_s.iloc[0],
            "end_epoch": df.cumul_time_s.iloc[-1]
        }
    )
    plot_trajectories.append(traj)

In [ ]:
# Predict speeds along each trajectory
busnetwork.update_travel_times(plot_trajectories, model)

In [ ]:
energy_results = []
consumptions = []
sig_errors = []

for i, traj in enumerate(plot_trajectories):
    # Prior to filtering or energy calculation; resample both aggregated and predicted cycles
    resampled_time = np.arange(traj.gdf['cumul_time_s'].min(), traj.gdf['cumul_time_s'].max(), 1)
    resampled_agg = np.interp(resampled_time, traj.gdf['cumul_time_s'].to_numpy(), traj.gdf['calc_speed_m_s'].to_numpy())
    resampled_pred = np.interp(resampled_time, traj.gdf['cumul_time_s'].to_numpy(), traj.gdf['pred_speed_m_s'].to_numpy())
    grade = spatial.divide_fwd_back_fill(np.diff(traj.gdf['calc_elev_m'], prepend=traj.gdf['calc_elev_m'].iloc[0]), traj.gdf['calc_dist_m'])
    resampled_grade = np.interp(resampled_time, traj.gdf['cumul_time_s'].to_numpy(), grade)

    # Energy analysis for aggregated
    cycle_agg = {
        "cycGrade": np.clip(resampled_grade, -0.15, 0.15),
        "mps": spatial.apply_sg_filter(resampled_agg, clip_min=0, clip_max=30),
        "time_s": resampled_time,
        "road_type": np.zeros(len(traj.gdf))
    }
    cycle_agg = fsim.cycle.Cycle.from_dict(fsim.cycle.resample(cycle_agg, new_dt=1))
    sim_drive_agg = fsim.simdrive.SimDrive(cycle_agg, veh)
    sim_drive_agg.sim_drive()

    # Energy analysis for predicted
    cycle_pred = {
        "cycGrade": np.clip(resampled_grade, -0.15, 0.15),
        "mps": spatial.apply_sg_filter(resampled_pred, clip_min=0, clip_max=30),
        "time_s": resampled_time,
        "road_type": np.zeros(len(traj.gdf))
    }
    cycle_pred = fsim.cycle.Cycle.from_dict(fsim.cycle.resample(cycle_pred, new_dt=1))
    sim_drive_pred = fsim.simdrive.SimDrive(cycle_pred, veh)
    sim_drive_pred.sim_drive()

    energy_results.append({"agg": (cycle_agg, sim_drive_agg), "pred": (cycle_pred, sim_drive_pred)})
    consumptions.append((sim_drive_agg.electric_kwh_per_mi, sim_drive_pred.electric_kwh_per_mi))
    sig_errors.append(spatial.eval_signal_error(sim_drive_agg.cyc.mps, sim_drive_agg.cyc.time_s, sim_drive_pred.cyc.mps, sim_drive_pred.cyc.time_s))

In [ ]:
fig, axes = plt.subplots(n_plot_routes, 2, figsize=(10,8))
for traj_n in range(n_plot_routes):
    ax = axes[traj_n,0]
    ax.legend().remove()
    ax.set_ylabel("Speed (mph)")
    ax.set_xlabel("Time (s)")
    ax.set_ylim(0,40)
    # Completely unprocessed trajectories
    sns.lineplot(x=plot_trajectories[traj_n].gdf["cumul_time_s"], y=plot_trajectories[traj_n].gdf["calc_speed_m_s"]*2.23, ax=ax, label="Aggregated")
    sns.lineplot(x=plot_trajectories[traj_n].gdf["cumul_time_s"], y=plot_trajectories[traj_n].gdf["pred_speed_m_s"]*2.23, ax=ax, label="Predicted")
for traj_n in range(n_plot_routes):
    ax = axes[traj_n,1]
    ax.set_ylabel("Speed (mph)")
    ax.set_xlabel("Time (s)")
    ax.set_ylim(0,40)
    sim_drive_agg, sim_drive_pred = energy_results[traj_n]["agg"][1], energy_results[traj_n]["pred"][1]
    # Trajectories post-processed, post-fastsim
    sns.lineplot(x=sim_drive_agg.cyc.time_s, y=sim_drive_agg.cyc.mph, ax=ax, label="Aggregated")
    sns.lineplot(x=sim_drive_pred.cyc.time_s, y=sim_drive_pred.cyc.mph, ax=ax, label="Predicted")
axes[0,0].set_title("Input Trip Cycles")
axes[0,1].set_title("Filtered Trip Cycles")

[ax.legend().remove() for ax in axes.flatten()]
axes[0,1].legend()
fig.suptitle(f"Example Drive Cycles for Aggregation/Prediction Models")
fig.tight_layout()
plt.show()
fig.savefig(Path("..","plots","agg_pred_cycles.png"))

In [ ]:
all_rmse = []
all_rmse_consumption = []
all_mean_consumption = []
for traj_n in range(len(energy_results)):
    sim_drive_agg, sim_drive_pred = energy_results[traj_n]["agg"][1], energy_results[traj_n]["pred"][1]
    error = [x-y for x,y in zip(sim_drive_agg.cyc.mph, sim_drive_pred.cyc.mph)]
    rmse_consumption = (sim_drive_agg.electric_kwh_per_mi - sim_drive_pred.electric_kwh_per_mi)**2
    all_rmse_consumption.append(rmse_consumption)
    all_mean_consumption.append((sim_drive_agg.electric_kwh_per_mi, sim_drive_pred.electric_kwh_per_mi))
    # sns.lineplot(error)
print(np.sqrt(np.mean(all_rmse_consumption)))
print(np.mean([x[0] for x in all_mean_consumption]))
print(np.mean([x[1] for x in all_mean_consumption]))